In [1]:
#Packages:
import pandas as pd
import string
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text

In [2]:
#maximum number of rows to display
pd.options.display.max_rows = 20

### Classes

In [3]:
class DbEngine():
    def __init__(self, user, password, ip, port, db):
        self.user = user
        self.password = password
        self.ip = ip
        self.port = port
        self.db = db
    
    def connect(self):
        return create_engine('postgresql://{}:{}@{}:{}/{}' \
                                .format(self.user, 
                                        self.password,
                                        self.ip,
                                        self.port,
                                        self.db), max_overflow=20)

In [4]:
class Queries():
    def __init__(self, engine):
        self.engine = engine
    
    def run(self, sql):
        result = self.engine.connect().execution_options(isolation_level="AUTOCOMMIT").execute((text(sql)))
        return pd.DataFrame(result.fetchall(), columns=result.keys())
    
    def insert(self, sql):
        return self.engine.connect().execution_options(isolation_level="AUTOCOMMIT").execute((text(sql)))

In [5]:
class ETL():
    def __init__(self, query_obj):
        self.queries = query_obj
        
    def inmate_static_info(self, data):
        ############## etnic recognition
        data_people = data[['INTERNOEN', 'GENERO', 'PAIS_INTERNO', 'REINCIDENTE', 
                            'ANO_NACIMIENTO', 'ESTADO_CIVIL','NIVEL_EDUCATIVO', 'CONDIC_EXPECIONAL']] \
                      .drop_duplicates(subset = ["INTERNOEN"])
        data_people = data_people.reset_index(drop=True)
        data_people['CONDIC_EXPECIONAL'] = data_people['CONDIC_EXPECIONAL'].fillna('NINGUNO')
        
        etnic_recognition = self.queries.run("""select * from reconocimiento_etnico""")
        data_people['reconocimiento_etnico'] = 'NINGUNO'
        for i in etnic_recognition.index:
            data_people.loc[(data_people['CONDIC_EXPECIONAL'].str.contains(etnic_recognition['nombre'].values[i])),
                            'reconocimiento_etnico'] =  etnic_recognition['nombre'].values[i]
            
        ############## foreign
        data_people['extranjero'] = 'N'
        data_people.loc[(data_people['CONDIC_EXPECIONAL'].str.contains('EXTRANJEROS')),'extranjero'] =  'S'
        
        ############## sexual diversity
        sexual_dive = self.queries.run("""select * from diversidad_sexual""")
        data_people['diversidad_sexual'] = 'N'
        for i in sexual_dive.index:
            data_people.loc[(data_people['CONDIC_EXPECIONAL'].str.contains(sexual_dive['nombre'].values[i])),
                            'diversidad_sexual'] =  'S'
    
        personas_genero = self.queries.run("""select * from persona where diversidad_sexual = 2""")
        for i in personas_genero.index:
            for j in sexual_dive.index:
                if sexual_dive['nombre'].values[j] in personas_genero['condicion_exepcional'].values[i]:
                    query = str(personas_genero['id_persona'].values[i]) + ', ' \
                                + str(sexual_dive['id_diversidad_sexual'].values[j])
                    self.queries.insert('INSERT INTO public.persona_diversidad_sexual (id_persona, id_diversidad_sexual) VALUES('+query+');')
        
        ############## people
        data_people.to_sql('personas_tmp', con=self.queries.engine)
        self.queries.run('select * from personas_tmp limit 5')
        self.queries.run('SELECT public.tcompararpersonas();')
        
    def inmate_variable_info(self, data):
        data_reg= data[['INTERNOEN', 'GENERO','DELITO','ESTADO_INGRESO','FECHA_CAPTURA',
                        'FECHA_INGRESO','ESTABLECIMIENTO','TENTATIVA','SUBTITULO_DELITO',
                        'AGRAVADO', 'CALIFICADO','FECHA_SALIDA','EDAD','DEPARTAMENTO', 'CIUDAD',
                        'ACTIVIDADES_TRABAJO', 'ACTIVIDADES_ESTUDIO', 'ACTIVIDADES_ENSEÑANZA',
                        'HIJOS_MENORES', 'CONDIC_EXPECIONAL','ESTADO','SITUACION_JURIDICA']]
        
        ############## No exceptional condition
        data_reg['CONDIC_EXPECIONAL'] = data_reg['CONDIC_EXPECIONAL'].fillna('NINGUNO')

        ############## Madre Gestantes
        data_reg['madre_gestante'] = 'NA'
        data_reg.loc[(data_reg['GENERO'].str.contains('FEMENINO')),'madre_gestante'] = 'N'
        data_reg.loc[(data_reg['CONDIC_EXPECIONAL'].str.contains('MADRE GESTANTE')),'madre_gestante'] =  'S'

        data_reg['madre_lactante'] = 'NA'
        data_reg.loc[(data_reg['GENERO'].str.contains('FEMENINO')),'madre_lactante'] = 'N'
        data_reg.loc[(data_reg['CONDIC_EXPECIONAL'].str.contains('MADRE LACTANTE')),'madre_lactante'] =  'S'

        ############## Discapacitad
        data_reg['discapacidad'] = 'N'
        data_reg.loc[(data_reg['CONDIC_EXPECIONAL'].str.contains('CON DISCAPACIDAD')),'discapacidad'] =  'S'

        ############## Adulto Mayor
        data_reg['adulto_mayor'] = 'N'
        data_reg.loc[(data_reg['CONDIC_EXPECIONAL'].str.contains('ADULTO MAYOR')),'adulto_mayor'] =  'S'
        
        ############## registro
        data_reg.to_sql('registros_tmp', con=self.queries.engine)
        self.queries.run('select * from registros_tmp limit 5')
        self.queries.run('SELECT public.tcompararreg();')

### Insert Data

#### Loading Data

In [6]:
# data reading from excel
data = pd.read_excel('data/ReincidenciaPospenadosNal201011Junio2020Rev.xlsx', skiprows = 6)
data.count()

INTERNOEN                142899
DELITO                   142899
TITULO_DELITO            142899
SUBTITULO_DELITO         142899
TENTATIVA                142899
                          ...  
ESTABLECIMIENTO          142899
DEPTO_ESTABLECIMIENTO    140108
MPIO_ESTABLECIMIENTO     140108
REGIONAL                 142899
ESTADO                   142899
Length: 32, dtype: int64

In [7]:
data.columns

Index(['INTERNOEN', 'DELITO', 'TITULO_DELITO', 'SUBTITULO_DELITO', 'TENTATIVA',
       'AGRAVADO', 'CALIFICADO', 'FECHA_INGRESO', 'FECHA_SALIDA',
       'FECHA_CAPTURA', 'SITUACION_JURIDICA', 'ANO_NACIMIENTO', 'EDAD',
       'GENERO', 'ESTADO_CIVIL', 'PAIS_INTERNO', 'DEPARTAMENTO', 'CIUDAD',
       'REINCIDENTE', 'ESTADO_INGRESO', 'ACTIVIDADES_TRABAJO',
       'ACTIVIDADES_ESTUDIO', 'ACTIVIDADES_ENSEÑANZA', 'NIVEL_EDUCATIVO',
       'HIJOS_MENORES', 'CONDIC_EXPECIONAL', 'CODIGO_ESTABLECIMIENTO',
       'ESTABLECIMIENTO', 'DEPTO_ESTABLECIMIENTO', 'MPIO_ESTABLECIMIENTO',
       'REGIONAL', 'ESTADO'],
      dtype='object')

#### ETL

In [8]:
db_engine = DbEngine(user = 'team77', 
                     password = 'mintic2020.',
                     ip = '172.17.0.2', 
                     port = '5432', 
                     db = 'minjusticia')
engine = db_engine.connect()

In [9]:
queries = Queries(engine)

In [10]:
etl = ETL(queries)

In [11]:
etl.inmate_static_info(data)
etl.inmate_variable_info(data)

/opt/conda/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/nlp/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand